## Group 3:
Classes: 3, 1, 4, 6, 8
    
    Open Country - 3
    Tall Building - 1
    Mountain - 4
    Highway - 6
    Coast - 8


In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import tarfile
import pandas as pd
import os
import re
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from sklearn.metrics import confusion_matrix
from io import StringIO
import pdb
from math import sqrt, log

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
class DatasetClass(Dataset):
    
    def __init__(self, folder, filename, label_dict):
        
        self.data = []
        self.filename = filename
        tar = tarfile.open(folder + '/' + filename)
        for file in tar.getmembers():
            f = tar.extractfile(file)
            if f != None:
                content = pd.read_csv(StringIO(f.read().decode()), sep=' ', header=None).values.ravel()
                self.data.append(content)
            
        self.y = torch.tensor(label_dict[self.filename[:-7]], dtype=torch.long)
    
    def __getitem__(self, idx):     
        
        return torch.tensor(self.data[idx], dtype=torch.float), self.y
      
    def __len__(self):
        
        return len(self.data)

In [4]:
def train_test_loader(directory, label_dict, train_fraction=0.8, num_workers=2, bs_fraction = 0.2):

    all_files = list(filter(lambda x: x.endswith('.tar.gz'), os.listdir(directory)))
    files = [file for file in all_files if file[:-7] in label_dict.keys()]
    
    datasets = list(map(lambda x : DatasetClass(directory, x, label_dict), files))
    dataset = ConcatDataset(datasets)
    N = dataset.cumulative_sizes[-1]
    
    train_size = int(N*train_fraction)
    test_size = N - train_size

    train_data, test_data = torch.utils.data.random_split(dataset, [train_size, test_size])

    trainloader = DataLoader(train_data, batch_size=bs_fraction*N, shuffle=True, num_workers=num_workers)
    testloader = DataLoader(test_data, batch_size=bs_fraction*N, shuffle=True, num_workers=num_workers)
    
    return trainloader, testloader

In [5]:
class GBRBM:
    
    def __init__(self, visible_nodes, h_len, lr_W=0.1, lr_bias=0.001):  
        # set lower lr for bias than for the weights
        self.N = visible_nodes.shape[0]
        v_len = visible_nodes.shape[1]
        self.V = visible_nodes.to(device)
        self.sigma2 = torch.var(self.V, 0)[0].to(device)  
        self.lr_W = lr_W
        self.lr_bias = lr_bias
        
        # Initialisation done based on the methods mentioned in the paper
        self.W = torch.empty(v_len, h_len).uniform_(-sqrt(6/(v_len+h_len)), sqrt(6/(v_len+h_len))).to(device)
        self.b = torch.mean(visible_nodes, axis=0).view(1,-1).to(device)
        self.c = torch.tensor([((torch.norm(self.b + self.W[:, i])**2 - torch.norm(self.b)**2)/(2*self.sigma2) +  log(0.01)).item() for i in range(h_len)]).view(1,-1).to(device)

    def get_h(self, v):
        
        a = torch.mm((v/self.sigma2).view(1,-1), self.W) + self.c
        f = torch.nn.Sigmoid()
        p_h_v = f(a)
        return p_h_v, torch.bernoulli(p_h_v)
    
    def get_v(self, h):
        a = torch.mm(h.view(1,-1), self.W.T) + self.b # mean of normal dist
        if (torch.isnan(a)).any().item():
            pdb.set_trace()
        else:
            pass
        v_h = torch.normal(mean=a, std=torch.sqrt(self.sigma2)).to(device)
        return v_h
    
    def params_update(self, p_h_v0, p_h_vk, v0, vk):
        self.W += self.lr_W*(torch.mm((v0/self.sigma2).view(-1,1), p_h_v0) - torch.mm((vk/self.sigma2).view(-1,1), p_h_vk))/self.N
        self.b += self.lr_bias*(v0 - vk)/self.N
        self.c += self.lr_bias*(p_h_v0 - p_h_vk)/self.N

        
    def one_epoch(self, k):
        for v0 in self.V:
            v_t = v0
            for t in range(k):  
                p_h_vt, h_t = self.get_h(v_t)
                if t==0:
                    p_h_v0 = p_h_vt                    
                v_t1 = self.get_v(h_t)
                v_t = v_t1

            try:
                V_k = torch.cat((V_k, v_t.view(1,-1)), dim=0)
                H_k = torch.cat((H_k, h_t.view(1,-1)), dim=0)
            except:
                V_k = v_t.view(1,-1)
                H_k = h_t.view(1,-1)
            self.params_update(p_h_v0, p_h_vt, v0, v_t)
        return V_k, H_k
        
    def train(self, k):
        ep = 0
        error_old = np.inf
        while True:
            ep += 1
            ## Check if error should be SSE?
            V_k, H_k = self.one_epoch(k)

            error_new = torch.sum((V_k - self.V)**2) 
            print('Epoch: {0}, Error: {1}, Error diff :{2}'.format(ep, error_new, error_old-error_new))
            
            if abs(error_new - error_old) <= 1e-6:
                print('Converged!')
                self.V_train = V_k
                self.H_train = H_k
                break
            error_old = error_new


In [6]:
class BBRBM:
    
    def __init__(self, visible_nodes, h_len, lr_W=0.01, lr_bias=0.001):
        
        # set lower lr for bias than for the weights
        
        self.N = visible_nodes.shape[0]        
        v_len = visible_nodes.shape[1]
        self.W = torch.randn(v_len, h_len).to(device)
        self.b = torch.randn(1, v_len).to(device)
        self.c = torch.randn(1, h_len).to(device)
        self.V = visible_nodes.to(device)
        self.lr_W = lr_W
        self.lr_bias = lr_bias        
        
    def get_h(self, v):
        
        a = torch.mm(v.view(1,-1), self.W) + self.c
        f = torch.nn.Sigmoid()
        p_h_v = f(a)
        return p_h_v, torch.bernoulli(p_h_v)
    
    def get_v(self, h):
        a = torch.mm(h.view(1,-1), self.W.T) + self.b
        f = torch.nn.Sigmoid()
        p_v_h = f(a)
        return p_v_h, torch.bernoulli(p_v_h)
    
    def params_update(self, p_h_v0, p_h_vk, v0, vk):
        self.W += self.lr_W*(torch.mm(v0.view(-1,1), p_h_v0) - torch.mm(vk.view(-1,1), p_h_vk))/self.N
        self.b += self.lr_bias*(v0 - vk)/self.N
        self.c += self.lr_bias*(p_h_v0 - p_h_vk)/self.N
        
    def one_epoch(self, k):

        for v0 in self.V:
            v_t = v0
            for t in range(k):  
                p_h_vt, h_t = self.get_h(v_t)
                if t==0:
                    p_h_v0 = p_h_vt                    
                p_v_ht, v_t1 = self.get_v(h_t)
                v_t = v_t1

            try:
                V_k = torch.cat((V_k, v_t.view(1,-1)), dim=0)
                H_k = torch.cat((H_k, h_t.view(1,-1)), dim=0)
            except:
                V_k = v_t.view(1,-1)
                H_k = h_t.view(1,-1)

            self.params_update(p_h_v0, p_h_vt, v0, v_t)

        return V_k
        
    def train(self, k):
        ep = 0
        error_old = np.inf
        while True:
            ep += 1
            ## Check if error should be SSE?
            V_k, H_h = self.one_epoch(k)
            error_new = torch.sum((V_k - self.V)**2) 
            print('Epoch: {0}, Error: {1}'.format(ep, error_new))
            
            if abs(error_new - error_old) <= 1e-6:
                print('Converged!')
                self.V_train = V_k
                self.H_train = H_k                
                break
            error_old = error_new

In [7]:
class FinalNet(nn.Module):
    
    def __init__(self, input_size, hidden_sizes, num_classes):
        super(FinalNet, self).__init__()
        
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.out = nn.Linear(hidden_sizes[2], num_classes)
    
    def forward(self, x):
        
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = self.out(x)
        
        return x
    
    def predict(self, X):
        
        with torch.no_grad():
            y_score = self.forward(X)
            y_pred = torch.argmax(y_score, axis=1)
            
        return y_pred   

In [8]:
def Gaussian_stacked_RBM(v, n_stacks, h_layers_len, learning_rates, k_list):
    '''
    Parameters:
    ------------
    v              - Input to RBM (visible nodes). Must be continuous valued, whitened.
    n_stacks       - No. of RMBs to be stacked
    h_layers_len   - List of no. of nodes in the hidden layer of each RMB
    learning_rates - List of list of learning rates (for Weights, bias) for each RBM
    k_list         - List of k values for each RBM
     '''
    
    weights = []
    biases = []
    
    print('------Gaussian Binary RBM------')
    gaussain = GBRBM(v_whitened.to(device), h_layers_len[0], learning_rates[0][0], learning_rates[0][1])
    gaussain.train(k_list[0])
    weights.append(gaussain.W)
    biases.append(gaussain.b)
    v_new = gaussain.H_train
    
    for i in range(1, n_stacks):
        print('------Binary Binary RBM {0}------'.format(i))
        binary = BBRBM(v_new.to(device), h_layers_len[i], learning_rates[i][0], learning_rates[i][1])
        binary.train(k[i])
        weights.append(binary.W)
        biases.append(binary.b)
        v_new = binary.H_train
        
    return weights, biases

1. Finding $\sigma^{2}$ for visible nodes
2. Whitening of data for GBRMB?
3. Choice of initial conditions



### Data pre-processing - Whitening the images

In [9]:
label_dict = {
    'tallbuilding': 1,
    'mountain': 4,
    'highway': 6,
    'coast': 8,
    'opencountry' : 3    
}

trainloader, testloader = train_test_loader('Data_Set_1(Colored_Images)', label_dict, train_fraction=0.8, num_workers=0, bs_fraction = 1)

In [10]:
v = list(trainloader)[0][0]
v_centered = v - torch.stack([torch.mean(v, 0)]*v.shape[0], dim=0)
cov = torch.mm(v_centered.T, v_centered)/v_centered.shape[0]
U, S, V = torch.svd(cov)
v_whitened = torch.mm(v_centered, U)/torch.sqrt(S)

In [13]:
n_stacks = 3
h_layers_len = [1000, 800, 600]
learning_rates = [[0.001, 0.0001]*3]
k_list = [100]*3

weights_pre_trained, biases_pre_trained = Gaussian_stacked_RBM(v_whitened, n_stacks, h_layers_len, learning_rates, k_list)

------Gaussian Binary RBM------
Epoch: 1, Error: 1976417.75, Error diff :inf
Epoch: 2, Error: 1979657.0, Error diff :-3239.25
Epoch: 3, Error: 1977993.0, Error diff :1664.0
Epoch: 4, Error: 1975409.625, Error diff :2583.375
Epoch: 5, Error: 1978228.75, Error diff :-2819.125
Epoch: 6, Error: 1976967.0, Error diff :1261.75
Epoch: 7, Error: 1976130.5, Error diff :836.5
Epoch: 8, Error: 1976587.0, Error diff :-456.5
Epoch: 9, Error: 1976408.625, Error diff :178.375
Epoch: 10, Error: 1978990.0, Error diff :-2581.375
Epoch: 11, Error: 1971500.5, Error diff :7489.5
Epoch: 12, Error: 1973394.75, Error diff :-1894.25
Epoch: 13, Error: 1974927.375, Error diff :-1532.625
Epoch: 14, Error: 1976651.75, Error diff :-1724.375
Epoch: 15, Error: 1975852.125, Error diff :799.625
Epoch: 16, Error: 1979080.25, Error diff :-3228.125
Epoch: 17, Error: 1976518.5, Error diff :2561.75
Epoch: 18, Error: 1978542.125, Error diff :-2023.625
Epoch: 19, Error: 1975020.0, Error diff :3522.125
Epoch: 20, Error: 198054

KeyboardInterrupt: 

In [ ]:
classifier = FinalNet(v.shape[1], h_layers_len, len(np.unique(np.array(trainloader)[0][1]))) 

In [ ]:
with torch.no_grad():
    
    classifier.fc1.weight.data = nn.Parameter(weights_pre_trained[0])
    classifier.fc1.bias.data = nn.Parameter(biases_pre_trained[0])
    
    classifier.fc2.weight = nn.Parameter(weights_pre_trained[1])
    classifier.fc2.bias = nn.Parameter(biases_pre_trained[1])
    
    classifier.fc3.weight = nn.Parameter(weights_pre_trained[2])
    classifier.fc3.bias = nn.Parameter(biases_pre_trained[2])

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(classifier.parameters(), lr=0.001, momentum=0.9)
classifier = classifier.to(device)

In [ ]:
trainloader, testloader = train_test_loader('Data_Set_1(Colored_Images)', label_dict, train_fraction=0.8, num_workers=0, bs_fraction = 0.2)

In [ ]:
old_loss = np.inf

max_epoch = 500

for epoch in range(max_epoch):

    running_loss = 0.0
    
    for data in trainloader:
        
        X, y = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()
        
        # Forward
        y_hat = classifier(X)
        
        # Calculate Loss (Cross Entropy)
        loss = criterion(y_hat, y)
        
        # Backpropagation
        loss.backward()
        
        # Update Parameters
        optimizer.step()
        
        running_loss += loss.item()
    
    print('Epoch', epoch+1, ': Loss =', running_loss)
    
    if abs(running_loss-old_loss)/running_loss < 1e-3:
        print('Converged')
        break
    
    old_loss = running_loss

print('Finished Training')

## ---------------------- End of code ---------------------- 

In [75]:
# vk = gaussian.train(100)

Epoch: 1, Error: 1973867.0, Error diff :inf
Epoch: 2, Error: 1975916.375, Error diff :-2049.375
Epoch: 3, Error: 1974804.625, Error diff :1111.75
Epoch: 4, Error: 1978021.375, Error diff :-3216.75


KeyboardInterrupt: 

In [73]:
# vk = gaussian.train(100)

Epoch: 1, Error: 1977611.875, Error diff :inf
Epoch: 2, Error: 1973322.0, Error diff :4289.875
Epoch: 3, Error: 1976089.25, Error diff :-2767.25
Epoch: 4, Error: 1976391.875, Error diff :-302.625
Epoch: 5, Error: 1975308.25, Error diff :1083.625
Epoch: 6, Error: 1975171.5, Error diff :136.75
Epoch: 7, Error: 1977419.125, Error diff :-2247.625
Epoch: 8, Error: 1978926.75, Error diff :-1507.625
Epoch: 9, Error: 1972214.5, Error diff :6712.25
Epoch: 10, Error: 1975139.0, Error diff :-2924.5
Epoch: 11, Error: 1972550.25, Error diff :2588.75
Epoch: 12, Error: 1974438.5, Error diff :-1888.25


KeyboardInterrupt: 

In [71]:
# vk = gaussian.train(100)

Epoch: 1, Error: 1974060.25, Error diff :inf
Epoch: 2, Error: 1977599.25, Error diff :-3539.0
Epoch: 3, Error: 1975388.75, Error diff :2210.5
Epoch: 4, Error: 1976873.25, Error diff :-1484.5
Epoch: 5, Error: 1971814.0, Error diff :5059.25
Epoch: 6, Error: 1974177.0, Error diff :-2363.0
Epoch: 7, Error: 1977052.5, Error diff :-2875.5
Epoch: 8, Error: 1973869.75, Error diff :3182.75
Epoch: 9, Error: 1974948.625, Error diff :-1078.875
Epoch: 10, Error: 1978998.125, Error diff :-4049.5
Epoch: 11, Error: 1977689.25, Error diff :1308.875
Epoch: 12, Error: 1977256.5, Error diff :432.75
Epoch: 13, Error: 1973529.25, Error diff :3727.25
Epoch: 14, Error: 1975417.75, Error diff :-1888.5
Epoch: 15, Error: 1973446.75, Error diff :1971.0
Epoch: 16, Error: 1977433.0, Error diff :-3986.25
Epoch: 17, Error: 1978050.75, Error diff :-617.75
Epoch: 18, Error: 1979904.5, Error diff :-1853.75
Epoch: 19, Error: 1977246.5, Error diff :2658.0
Epoch: 20, Error: 1976824.25, Error diff :422.25
Epoch: 21, Error: 

KeyboardInterrupt: 

In [60]:
# v_len = 828
# h_len = 1000
# sigma2 = torch.var(v_whitened, 0)
# W = torch.empty(v_len, h_len).uniform_(-sqrt(6/(v_len+h_len)), sqrt(6/(v_len+h_len))).to(device)
# b = torch.mean(v_whitened, axis=0).view(1,-1).to(device)
# c = torch.tensor([((torch.norm(b + W[:, i])**2 - torch.norm(b)**2)/(2*torch.var(v_whitened)) +  log(0.01)).item() for i in range(h_len)]).view(1,-1).to(device)